#### Kaggle quora

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import bcolz
import pickle
import re
from numpy.random import random, permutation, randn, normal, uniform, choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2,l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import deserialize as layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import SpatialDropout1D


Using TensorFlow backend.


In [2]:
train_data=pd.read_csv('./data/train.csv')
test_data=pd.read_csv('./data/test.csv')


In [3]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
test_data.tail()

,test_id,question1,question2
2345791,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
2345792,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
2345793,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
2345794,2345794,What were the best and worst things about publ...,What are the best and worst things examination...
2345795,2345795,What is the best medication equation erectile ...,How do I out get rid of Erectile Dysfunction?


In [5]:
train_data[train_data['is_duplicate']==1]['id'].count(),train_data[train_data['is_duplicate']==0]['id'].count()

(149263, 255027)

In [6]:
labels= train_data['is_duplicate']

In [7]:
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

In [8]:
from collections import Counter

total_counts =Counter()
# bag of words here
for idx, row in train_data.iterrows():
   
       total_counts.update(str(row['question1']).lower().split(' '))
       total_counts.update(str(row['question2']).lower().split(' '))
    
print("Total words in data set: ", len(total_counts))

Total words in data set:  201112


In [9]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)
print(vocab[:60])

['the', 'what', 'is', 'how', 'i', 'a', 'to', 'in', 'of', 'do', 'are', 'and', 'can', 'for', 'you', 'why', 'my', 'best', 'it', 'on', 'does', 'which', 'if', 'or', 'be', 'some', 'that', 'have', 'should', 'with', 'get', 'an', 'your', 'from', 'will', 'when', 'who', 'at', 'good', 'people', 'would', 'like', 'there', 'as', 'between', 'about', 'most', 'we', 'one', 'make', 'did', 'where', 'way', 'by', 'not', 'any', 'was', 'india?', 'after', 'they']


In [10]:
print(vocab[-1], ': ', total_counts[vocab[-1]])

"certified :  1


In [11]:
word22idx = {word:i  for i,word in enumerate(vocab)}

In [12]:
def text_to_vector(text):
    text=str(text).lower()
    words=[]
    for token in text.split(' '):
        idx=word22idx.get(token, None)
        if idx== None:
            continue
        else:
            words.append(idx)
    return words

In [13]:
idx2word = {word:i  for word,i in enumerate(vocab)}

In [14]:
train_data['question1_idx']=train_data['question1'].apply(text_to_vector)

In [15]:
train_data['question2_idx']=train_data['question2'].apply(text_to_vector)

In [16]:
# train_list=[]
# for g, df in train_data.groupby(np.arange(len(train_data)) // 10000):
#     train_list.append(df)

In [17]:
# len(train_list)
vocab_size = 60000

In [18]:
# vector_list=[]
train_data['question1_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_data['question1_idx']]
train_data['question2_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_data['question2_idx']]

In [19]:
train_data['question1_idx_trun'][0]

array([   1,    2,    0, 1394,   53, 1394, 2946,    6,  518,    7,  747,
        565,    7,   57])

In [20]:
# for i in range(len(train_list)):
#     word_vectors = np.zeros((len(train_list[i]),2, len(vocab)), dtype=np.int_)
#     for ii,row in train_list[i].iterrows():
#             word_vectors[ii-i*10000][0] = text_to_vector(str(row['question1']))
#             word_vectors[ii- i*10000][1] = text_to_vector(str(row['question2']))
#     vector_list.append(word_vectors)
#     del word_vectors

In [21]:
def load_array(fname):
    return bcolz.open(fname)[:]
def load_glove(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'),encoding='utf-8'),
        pickle.load(open(loc+'_idx.pkl','rb'),encoding='utf-8'))

In [22]:
vecs, words, wordidx = load_glove('./6B.300d/6B.300d')

In [23]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((len(vocab), n_fact))

    for i in range(len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx.get(word, None)
            if src_idx is None:
                emb[i] = normal(scale=0.6, size=(n_fact,))
            else:
                    
                emb[i] = vecs[src_idx]
            
        else:
             
            # If we can't find the word in glove, randomly initialize
             emb[i] = normal(scale=0.6, size=(n_fact,))
            

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [24]:
emb = create_emb()

In [25]:
x_train= train_data[['question1_idx_trun','question2_idx_trun']]

In [26]:
x_train.head()

,question1_idx_trun,question2_idx_trun
0,"[1, 2, 0, 1394, 53, 1394, 2946, 6, 518, 7, 747...","[1, 2, 0, 1394, 53, 1394, 2946, 6, 518, 7, 747..."
1,"[1, 2, 0, 748, 8, 18774, 40953, 13502]","[1, 40, 172, 22, 0, 80, 240, 14004, 0, 18774, ..."
2,"[3, 12, 4, 177, 0, 466, 8, 16, 521, 2560, 164,...","[3, 12, 521, 466, 24, 4265, 53, 2120, 179, 35144]"
3,"[15, 65, 4, 3020, 252, 5387, 3, 12, 4, 545, 111]","[78, 0, 4543, 35, 59999, 2, 2513, 53, 59999]"
4,"[21, 48, 8361, 7, 274, 59918, 32642, 35951, 20...","[21, 2370, 40, 1358, 7, 2320, 810]"


In [27]:
# x_train = pd.DataFrame.from_records(vector_list[0])
# for i in range(1, len(vector_list)):
#     x_train=pd.concat([x_train, pd.DataFrame.from_records(vector_list[i])], ignore_index=True)


In [28]:
train_data.shape,x_train.shape

((404290, 10), (404290, 2))

In [29]:
lens = np.array(map(len, x_train))


In [30]:
x_train.shape

(404290, 2)

In [31]:
seq_len = 100

trn = sequence.pad_sequences(x_train['question1_idx_trun'], maxlen=seq_len, value=0)
trn2 = sequence.pad_sequences(x_train['question2_idx_trun'], maxlen=seq_len, value=0)


In [72]:
X=np.hstack((trn,trn2))

In [74]:
X.shape

(404290, 200)

In [75]:
emb.shape

(201112, 300)

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.23, random_state=42)

In [77]:
model = Sequential([
    Embedding(len(emb), 300, input_length=200, dropout=0.2, 
              weights=[emb], trainable=False),
    SpatialDropout1D(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

C:\Users\NDIRANGU\Anaconda3\envs\py35\lib\site-packages\keras\legacy\interfaces.py:123: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  warnings.warn('The `dropout` argument is no longer support in `Embedding`. '
C:\Users\NDIRANGU\Anaconda3\envs\py35\lib\site-packages\keras\legacy\interfaces.py:86: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
  '` call to the Keras 2 API: ' + signature)


In [78]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=2, batch_size=64)

C:\Users\NDIRANGU\Anaconda3\envs\py35\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 311303 samples, validate on 92987 samples
Epoch 1/2
311296/311303 [============================>.] - ETA: 0s - loss: 0.5432 - acc: 0.7210    

In [84]:
X_train.shape

(311303, 2)